# Hyperparameter Optimization with aqua-blue

Prediction accuracy is highly correlated to the hyperparameters of a reservoir computer. The `hyper_opt` module provides functionality to optimize hyperparameters through random search algorithms. **Note that this is an experimental feature and is subject to change. Any feedback is appreciated.**

## Aqua-blue Hyperparameters
Aqua-blue models have 4 hyperparameters with significant effect on prediction accuracy. 

### Spectral Radius ($r_s$)
The spectral radius is the largest absolute eigenvalue of the reservoir weight matrix, $W_{res}$. It controls the echo state property of the system, ensuring that memory of past states gradually fades over time. The approximate ranges which define behavior for general use cases are given below, however the best way to determine the optimal $s_r$ for a task is using an optimization algorithm.
$$r_s < 1 \to \text{ stable dynamics, past states fade quickly}$$
$$r_s \approx 1 \to \text{ balance between past states and current input}$$ 
$$r_s > 1 \to \text{ chaotic dynamics, could lead to numerical instability }$$

### Sparsity
The sparsity is the proportion of zero entries in the reservoir weight matrix $W_{res}$. It determines the extent of connection between neurons. Higher sparsity values lead to reduced computation at the cost of higher overfitting risk. Lower sparsity values increase neuron connection density, which leads to more complex dynamics at the price of higher computational cost.

### Leaking Rate ($\alpha$) 
The leaking rate controls the priority of past states versus current states. At each timestep, 
    $$y_t = (1 - \alpha) y_{t-1} + \alpha f(W_\text{in} x_t + W_\text{res} y_{t-1})$$
where $y_t$ and $y_{t-1}$ are reservoir states and $x_t$ is an input state. Therefore, the leaking rate is directly proportional to how fast the model updates.

### rcond 
The output later is trained using ridge regression, which involves computing the Moore-Penrose pseudo-inverse. `rcond` is a cutoff for small singular values, which is used to improve numerical stability.